모델을 부분적으로 로딩하거나 부분 모델을 로딩하는 것은 새로운 복잡한 모델을 전이 학습하거나 학습할 때 일반적인 시나리오다. 소수의 매개 변수만 사용할 수 있더라도 학습된 파라미터들을 잘 활용하면 warmstart에 유용하게 쓰이며, 모델이 처음부터 학습하는 것보다 훨씬 빠르게 converge되는데 도움이 될 것입니다.

## 개요
일부 키가 없는 부분 ```state_dict```에서 로드하든, 로드 중인 모델보다 더 많은 키로 ```state_dict```을 로드하든 상관없이 ```load_state_dict()``` 함수에서 strict argument를 False로 설정하여 매칭되지않은 키를 무시할 수 있다. 이 레시피에서는, 다른 모델의 파라미터를 사용하여 모델을 warmstarting 하는 실험을 한다.


## 단계
1. 데이터를 불러올 때 필요한 라이브러리들 불러오기
2. 신경망 A와 B를 정의
3. 모델 A 저장
4. 모델 B에 로드

### 1. 데이터를 불러올 때 필요한 라이브러리들 불러오기

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

### 2. 신경망 A와 B를 정의

In [3]:
class NetA(nn.Module):
    def __init__(self):
        super(NetA, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        return x
    
netA = NetA()

class NetB(nn.Module):
    def __init__(self):
        super(NetB, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
netB = NetB()

### 3. 모델 A 저장

In [4]:
PATH = "model.pt"

torch.save(netA.state_dict(), PATH)

### 4. 모델 B에 로드
한 layer에서 다른 layer로 파라미터를 로드하지만 일부 키가 일치하지 않는 경우 로드 중인 state_dict 매개변수 키 이름을 로드 중인 모델의 키와 일치하도록 변경하기만 하면 된다.

In [5]:
netB.load_state_dict(torch.load(PATH), strict=False)

<All keys matched successfully>